In [12]:
# Bitcoin Analysis and Visualization Script - Version 013
# This version includes:
# - Bar charts for metrics visualization with price on left y-axis and indicators on right y-axis
# - Heatmap for overall sentiment analysis with improved styling
# - Removal of grid lines from all charts
# - Adjusted and aligned heatmap color legend
# - Saving all charts to a single HTML file with white background

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2020, 1, 1).timestamp())  # Jan 1, 2023
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/market/spot_cvd_sum',
    'https://api.glassnode.com/v1/metrics/market/spot_volume_daily_sum'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

def calculate_momentum_rsi(df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True):
    price_change = df[column].diff()
    gains = price_change.where(price_change > 0, 0)
    losses = -price_change.where(price_change < 0, 0)
    avg_gains = gains.rolling(window=rsi_window, min_periods=1).mean()
    avg_losses = losses.rolling(window=rsi_window, min_periods=1).mean()
    relative_strength = avg_gains / avg_losses
    rsi = 100 - (100 / (1 + relative_strength))

    if normalize:
        rsi_min = rsi.rolling(window=window_norm, min_periods=1).min()
        rsi_max = rsi.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (rsi - rsi_min) / (rsi_max - rsi_min) - 1
        return normalized_momentum
    else:
        return rsi

def calculate_spot_cvd_bias(df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True):
    rolling_sum = df[column].rolling(window=window_sum).sum()
    
    if normalize:
        rolling_min = rolling_sum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = rolling_sum.rolling(window=window_norm, min_periods=1).max()
        normalized_bias = 2 * (rolling_sum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_bias
    else:
        return rolling_sum

def calculate_spot_volume_momentum(df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True):
    fast_ma = df[column].rolling(window=fast_window).mean()
    slow_ma = df[column].rolling(window=slow_window).mean()
    volume_momentum = fast_ma / slow_ma

    if normalize:
        rolling_min = volume_momentum.rolling(window=window_norm, min_periods=1).min()
        rolling_max = volume_momentum.rolling(window=window_norm, min_periods=1).max()
        normalized_momentum = 2 * (volume_momentum - rolling_min) / (rolling_max - rolling_min) - 1
        return normalized_momentum
    else:
        return volume_momentum

# Apply the functions to our merged_df
merged_df['Price Momentum'] = calculate_momentum_rsi(merged_df, column='price_usd_close', rsi_window=14, window_norm=90, normalize=True)
merged_df['Spot CVD Bias'] = calculate_spot_cvd_bias(merged_df, column='spot_cvd_sum', window_sum=7, window_norm=90, normalize=True)
merged_df['Spot Volume Momentum'] = calculate_spot_volume_momentum(merged_df, column='spot_volume_daily_sum', fast_window=7, slow_window=90, window_norm=90, normalize=True)

In [13]:
# ... (previous code remains the same)

# Visualization section

def create_indicator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=365)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator,
            name=indicator_column,
            line=dict(color='green', width=2),
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add red color for negative values
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color='red', width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 3, 5, 7, 9, 11]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]
    last_date = indicator.index[-1]

    # Determine the color based on the last value
    indicator_color = 'green' if last_value >= 0 else 'red'

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        xaxis_title={
            'text': "Date",
            'font': {'color': 'black', 'size': 18}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

# Assuming merged_df is your dataframe containing all the necessary data
# If you haven't created merged_df yet, you need to do that before this visualization part

# Create charts for each indicator
indicators = [
    ('Price Momentum', "Bitcoin: Price Momentum"),
    ('Spot CVD Bias', "Bitcoin: Spot CVD Bias"),
    ('Spot Volume Momentum', "Bitcoin: Spot Volume Momentum")
]

for indicator, title in indicators:
    fig = create_indicator_chart(merged_df, indicator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_last_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [14]:
merged_df.tail()

,price_usd_close,spot_cvd_sum,spot_volume_daily_sum,Price Momentum,Spot CVD Bias,Spot Volume Momentum
t,,,,,,
2024-09-23,63322.551391,8.604391e+07,8.047968e+09,0.653095,0.574273,-0.408427
2024-09-24,64338.712500,-1.330203e+08,7.888459e+09,0.679914,0.383670,-0.474315
2024-09-25,63078.050802,-1.022103e+08,6.941465e+09,0.516619,0.199999,-0.599471
2024-09-26,65134.209238,2.468616e+08,1.020946e+10,0.589071,0.417909,-0.647210
2024-09-27,65767.337050,9.809854e+07,9.048639e+09,0.481533,0.540986,-0.645320
